**Examen FINAL - Robótica II**

**Tema: Detector de Frutas**

**Integrantes**
- Ricardo Romero.
- Iván González.
- José Bergottini.

**Código de control**

In [ ]:
!pip install pyserial
# LIBRERIAS: Se importan las librerias requeridas
from tensorflow.keras.utils import load_img, img_to_array
from IPython.display import display, Javascript, clear_output
from PIL import Image as PILImage
from base64 import b64decode
from keras.models import load_model
from io import BytesIO
from mpl_toolkits.mplot3d import axes3d
from scipy.spatial.transform import Rotation as R
import numpy as np
import serial
import random
import pandas as pd
import cv2
import time
import tkinter as tk
#import ThorLib as th
import matplotlib.pyplot as plt


%matplotlib inline

import tensorflow as tf
print(tf.__version__)

# Funciones de la librería THOR

In [ ]:
def CineD(q):
    #Agregamos las librerias que vamos a emplear
    import numpy as np
    # Variables de modelado DH:
    offset=np.array([0,0,np.pi/2,0,0,0]) #  Para el modelo acostado
    d=np.array([0.202,0,0,0.195,0,0.06715])
    a=np.array([0,0.160,0,0,0,0])
    alfa=np.array([np.pi/2,0,np.pi/2,-np.pi/2,np.pi/2,0])
    teta=q+offset
    T=np.eye(4,4)
    for i in range(6):
        A=np.array([[(np.cos(teta[i])),-np.cos(alfa[i])*np.sin(teta[i]),np.sin(alfa[i])*np.sin(teta[i]),a[i]*np.cos(teta[i])],
            [np.sin(teta[i]),np.cos(alfa[i])*np.cos(teta[i]),-np.sin(alfa[i])*np.cos(teta[i]),a[i]*np.sin(teta[i])],
            [0,np.sin(alfa[i]),np.cos(alfa[i]),d[i]],
            [0,0,0,1]])
        T=T@A
    return T


def CineI(T):
    import numpy as np
    haySolucion=1;
    Codo=1
    P=T[0:3,3]  #Posicion de la herramienta
    na=T[0:3,2]  #Vector de aproximacion
    o=T[0:3,1]  #Vector normal
    Pm=P-0.06715*na

    V1=np.array([Pm[0],Pm[1],0])
    if V1[0]==0 and V1[1]==0:
        v1=np.array([0,0,0])
    else:
        v1=V1/np.sqrt(np.dot(V1,V1))   #Definicion del versor V1 si V1 no es nulo

    if v1[1]<0:
        q1=-np.arccos(v1[0]) #Solucion para 3er y 4to cuadrante
    elif(v1[0]==0 and v1[1]==0):
        q1= np.NaN   #Solucion indeterminada
    else:
        q1=np.arccos(v1[0]) #Solucion para 1er y segundo cuadrante
        #print(180*q1/np.pi)
    #[P_mx Cq_1+P_my Sq_1 0 P_mz]
    Pmpi=np.array([Pm[0]*v1[0] + Pm[1]*v1[1], 0, Pm[2]])
    # u_1=P_mπz-0.202
    u1=Pmpi[2] - 0.202
    # v=〖0.160〗^2-〖0.195〗^2+P_mπx^2+u_1^2
    v=0.160**2 - 0.195**2 + Pmpi[0]**2+u1**2
    #A=4u_1^2+4P_mπx^2
    A=4*u1**2+4*Pmpi[0]**2
    #B=-4vu_1
    B=-4*v*u1
    #C=v^2-4*〖0.160〗^2 P_mπx^2
    C=v**2-4*0.160**2*Pmpi[0]**2
# Si el discriminante de la cuadratica es negativa significa que no existe solucion

    if np.round(B**2-4*A*C,6)<0:
        q=np.zeros((4,6))
        haySolucion=0
        return [q, haySolucion, Codo]
    else:
        u=np.array([(-B+np.sqrt(np.round(B**2-4*A*C,6)))/(2*A),(-B-np.sqrt(np.round(B**2-4*A*C,6)))/(2*A)])
        if (0.160**2-u[0]**2)<0:
            Codo=0
           # x_π=√(〖0.160〗^2-u^2 )  0  z_π=u+0.202
            P2pi=np.array([[np.sqrt(0.160**2-u[1]**2), 0, u[1]+0.202],
                            [np.sqrt(0.160**2-u[1]**2), 0, u[1]+0.202]])
        elif (0.160**2-u[1]**2)<0:
            P2pi=np.array([[np.sqrt(0.160**2-u[0]**2), 0, u[0]+0.202],
                            [np.sqrt(0.160**2-u[0]**2), 0, u[0]+0.202]])
            Codo=0
        elif (0.160**2-u[0]**2)<0 and np.sqrt(0.160**2-u[1]**2)<0:
            q=np.zeros((4,6))
            haySolucion=0
            Codo=0
            return [q, haySolucion, Codo]
        else:
            P2pi=np.array([[np.sqrt(0.160**2-u[0]**2), 0, u[0]+0.202],
                            [np.sqrt(0.160**2-u[1]**2), 0, u[1]+0.202]])
        #P_2b=R_z (q_1 ) P_(2π )
        P2=np.array([[v1[0], -v1[1], 0],
                    [v1[1], v1[0],0],
                    [0,0,1]]) @P2pi.T
        #Valores de q2

        V2=np.zeros((2,3))
        v2=np.zeros((2,3))
        q2=np.zeros(2)
        V2[0,:]=P2[:,0]-np.array([[0,0,0.202]]) #Vector V2
        V2[1,:]=P2[:,1]-np.array([[0,0,0.202]])
        v2[0,:]=V2[0,:]/np.sqrt(V2[0,:]@V2[0,:])
        q2[0]=np.pi/2-np.arccos(v2[0,2])
        v2[1,:]=V2[1,:]/np.sqrt(V2[1,:]@V2[1,:])
        q2[1]=np.pi/2-np.arccos(v2[1,2])

        #Valores de q3
        V3=np.zeros((2,3))
        v3=np.zeros((2,3))
        q3=np.zeros(2)

        vpi=np.cross(v1,np.array([0,0,1]))
        V3[0,:]=Pm-P2[:,0]
        V3[1,:]=Pm-P2[:,1]
        v3[0,:]=V3[0,:]/np.sqrt(V3[0,:]@V3[0,:])
        v3[1,:]=V3[1,:]/np.sqrt(V3[1,:]@V3[1,:])
        v23=np.cross(v2[0,:],v3[0,:])
        if np.dot(v23,vpi)>0:
            q3[0]=np.arccos(np.round(np.dot(v2[0,:],v3[0,:]),6))
        else:
            #print(np.dot(v2[0,:],v3[0,:]))
            q3[0]=-np.arccos(np.round(np.dot(v2[0,:],v3[0,:]),6))

        v23=np.cross(v2[1,:],v3[1,:])
        if np.dot(v23,vpi)>0:
            q3[1]=np.arccos(np.round(np.dot(v2[1,:],v3[1,:]),6))
        else:
            q3[1]=-np.arccos(np.round(np.dot(v2[1,:],v3[1,:]),6))

#Seleccion de solucion codo arriba: El que tenga menor valor de q3
#Entonces q3 tiene la forma q3[q3carr, q3cabaj]
        if q3[0]>q3[1]:
            band=q3[0]
            q3[0]=q3[1]
            q3[1]=band

            band=q2[0]
            q2[0]=q2[1]
            q2[1]=band

# Soluciones para los valores de q4 q5 y q6
#V_α=V ̂_3×(a ̂×V ̂_3 )
        q4=np.zeros(2)
        q5=np.zeros(2)
        v4=np.zeros((2,3))
        Valpha=np.cross(np.cross(na,v3[0,:]),v3[0,:])
        valpha=Valpha/np.sqrt(Valpha@Valpha)
        v4[0,:]=np.cross(vpi,v3[0,:])
        #print(np.round(np.dot(valpha,vpi),6))
        if np.round(np.dot(valpha,vpi),6):
            q4[0]=0
            q4[1]=0
        elif np.round(np.dot(valpha,vpi),6)>0:
            q4[0]=np.arccos(np.round(np.dot(v4[0,:],valpha),6))
            if q4[0]<0:
                q4[1]=(np.pi+q4[0])
            else:
                q4[1]=-(np.pi-q4[0])
        elif np.round(np.dot(valpha,vpi),6)<0:
            q4[0]=-np.arccos(np.round(np.dot(v4[0,:],valpha),6))
            if q4[0]<0:
                q4[1]=(np.pi+q4[0])
            else:
                q4[1]=-(np.pi-q4[0])

    #Calculo de q5--------------------------------------------------------------------

        if q4[0]>0 and np.dot(valpha,vpi)>0:
            q5[0]=-np.arccos(np.dot(na,v3[0,:]))
        elif q4[0]<0 and np.dot(valpha,vpi)>0:
            q5[0]=np.arccos(np.dot(na,v3[0,:]))
        elif q4[0]>0 and np.dot(valpha,vpi)<0:
            q5[0]=np.arccos(np.dot(na,v3[0,:]))
        elif q4[0]<0 and np.dot(valpha,vpi)<0:
            q5[0]=- np.arccos(np.dot(na,v3[0,:]))
        elif q4[0]==0 or np.dot(valpha,vpi)==0:
            q5[0]=np.arccos(np.dot(na,v3[0,:]))
        else:
            q5[0]=np.NaN
        q5[1]=-q5[0]


    #Calculo de q6---------------------------------------------------------------
        q6=np.zeros(2)
        Vbeta=np.cross(v3[0,:],na)
        vbeta=Vbeta/np.sqrt(np.dot(Vbeta,Vbeta))
        q6i=np.arccos(np.round(np.dot(vbeta,o),6))
        band=np.dot(o,valpha)
        #print(q6i)
    #Obs: Al no encontrar una condicion valida para todos los puntos se recurrio a apciones de fuerza bruta
        if np.round(q4[0],6)>0 and np.round(q5[0],6)>0 and band>0:
            q6[0]=q6i
        elif np.round(q4[0],6)>0 and np.round(q5[0],6)>0 and band<0:
            q6[0]=-q6i
        elif np.round(q4[0],6)<0 and np.round(q5[0],6)>0 and band>0:
            q6[0]=q6i
        elif np.round(q4[0],6)<0 and np.round(q5[0],6)>0 and band<0:
            q6[0]=-q6i
        elif np.round(q4[0],6)>0 and np.round(q5[0],6)<0 and band>0:
            q6[0]=-(np.pi-q6i)
        elif np.round(q4[0],6)>0 and np.round(q5[0],6)<0 and band<0:
            q6[0]=(np.pi-q6i)
        elif np.round(q4[0],6)<0 and np.round(q5[0],6)<0 and band>0:
            q6[0]=-(np.pi-q6i)
        elif np.round(q4[0],6)<0 and np.round(q5[0],6)<0 and band<0:
            q6[0]=(np.pi-q6i)
        elif np.round(q4[0],6)==0 and np.round(q5[0],6)>0 and band>0:
            q6[0]=q6i
        elif np.round(q4[0],6)==0 and np.round(q5[0],6)>0 and band<0:
            q6[0]=-q6i
        elif np.round(q4[0],6)==0 and np.round(q5[0],6)<0 and band>0:
            q6[0]=-(np.pi-q6i)
        elif np.round(q4[0],6)==0 and np.round(q5[0],6)<0 and band<0:
            q6[0]=(np.pi-q6i)


        if np.round(q4[1],6)>0 and np.round(q5[1],6)>0 and band>0:
            q6[1]=q6i
        elif np.round(q4[1],6)>0 and np.round(q5[1],6)>0 and band<0:
            q6[1]=-q6i
        elif np.round(q4[1],6)<0 and np.round(q5[1],6)>0 and band>0:
            q6[1]=q6i
        elif np.round(q4[1],6)<0 and np.round(q5[1],6)>0 and band<0:
            q6[1]=-q6i
        elif np.round(q4[1],6)>0 and np.round(q5[1],6)<0 and band>0:
            q6[1]=-(np.pi-q6i)
        elif np.round(q4[1],6)>0 and np.round(q5[1],6)<0 and band<0:
            q6[1]=(np.pi-q6i)
        elif np.round(q4[1],6)<0 and np.round(q5[1],6)<0 and band>0:
            q6[1]=-(np.pi-q6i)
        elif np.round(q4[1],6)<0 and np.round(q5[1],6)<0 and band<0:
            q6[1]=(np.pi-q6i)
        elif np.round(q4[1],6)==0 and np.round(q5[1],6)>0 and band>0:
            q6[1]=q6i
        elif np.round(q4[1],6)==0 and np.round(q5[1],6)>0 and band<0:
            q6[1]=-q6i
        elif np.round(q4[1],6)==0 and np.round(q5[1],6)<0 and band>0:
            q6[1]=-(np.pi-q6i)
        elif np.round(q4[1],6)==0 and np.round(q5[1],6)<0 and band<0:
            q6[1]=(np.pi-q6i)


        if abs(q4[0])<abs(q5[0]):
            q=np.array([[q1,q2[0],q3[0],q4[0],q5[0],q6[0]],
                      [q1,q2[0],q3[0],q4[1],q5[1],q6[1]],
                      [q1,q2[1],q3[1],q4[0],q5[0],q6[0]],
                      [q1,q2[1],q3[1],q4[1],q5[1],q6[1]]])
        else:
            q=np.array([[q1,q2[0],q3[0],q4[1],q5[1],q6[1]],
                      [q1,q2[0],q3[0],q4[0],q5[0],q6[0]],
                      [q1,q2[1],q3[1],q4[1],q5[1],q6[1]],
                      [q1,q2[1],q3[1],q4[0],q5[0],q6[0]]])

        q[np.isnan(q)] = 0 # Agrego esta línea
        return [q, haySolucion, Codo]

# Work Package 2

In [ ]:
def TPosition(pos_x,pos_y,pos_z,ang_z1,ang_y,ang_z2):
    # Pasa los ángulos de euler a la matriz de rotación
    r = R.from_euler('zyz', [ang_z1, ang_y, ang_z2], degrees=True) #yaw, pitch, yaw
    Rot=r.as_matrix()

    # Vector de posiciones
    Pos=np.array([pos_x,pos_y,pos_z])

    # Se crea la matriz de transformación homogenea
    T=np.eye(4,4)
    T[:3,:3]=Rot
    T[0:3,3]=Pos
    Tn= np.array([T])
    [qn, t, valid] = solution_generation(Tn)


    qs = np.zeros([len(qn), 6]) # Elige una de las 4 soluciones para las n matrices

    if (valid.sum() == 0):
        print("No existen soluciones para este punto")
        return None

    # Está preparado para trabajar con un punto
    if (len(qn) == 1):
        index = -1
        for i in range(4):
            if valid[i] == 1:
                index = i
                break
        qs = qn[:, index, :]
    else:
        print("Entrada no válida")
        return None

    return [qs, t, T]

In [ ]:
def WPosition(pos_x,pos_y,pos_z,ang_z1,ang_y,ang_z2):

    # Pasa los ángulos de euler a la matriz de rotación
    r = R.from_euler('zyz', [ang_z1, ang_y, ang_z2], degrees=True) #yaw, pitch, yaw
    Rot=r.as_matrix()

    # Vector de posiciones
    Pos=np.array([pos_x,pos_y,pos_z])

    # Se crea la matriz de transformación homogenea
    T=np.eye(4,4)
    T[:3,:3]=Rot
    T[0:3,3]=Pos
    Tn= np.array([T])
    [qn, t, valid] = solution_generation(Tn)


    qs = np.zeros([len(qn), 6]) # Elige una de las 4 soluciones para las n matrices

    if (valid.sum() == 0):
        print("No existen soluciones para este punto")
        return None

    # Está preparado para trabajar con un punto
    if (len(qn) == 1):
        index = -1
        for i in range(4):
            if valid[i] == 1:
                index = i
                break
        qs = qn[:, index, :]
    else:
        print("Entrada no válida")
        return None

    Wp = T[:3,3] - 0.06715*T[:3,2]

    return [qs, t, Wp]

# Funciones

In [ ]:
def enviar_trama(qs, Label, tsol = np.array([0])):
    # qs: valores angulares de las articulaciones en radianes
    global ard

    print(f"Va a la posición: {Label}")

    q = np.round(qs*180/np.pi, 2)

    # Se multiplica por un factor para enviar la trama
    q1 = round(q[0]*(1/15),2)
    q2 = round(q[1]*(1/18),2)
    q3 = round(q[2]*(-1/18),2)
    q4 = round(q[3]*(-1/18),2)
    q5 = round(q[4]*(1/300),2)
    q6 = round(q[5]*(-2/225),2)

    # Trama a enviar
    grbl_string = f"G1 A{q1} B{q2} C{q2} D{q3} X{q4} Y{q6+q5} Z{q6-q5} F50\n"

    # Se envía la trama
    ard.write((grbl_string).encode('UTF-8'))

    answer = 'no'.encode('utf-8')

    while(answer != b'ok\r\n'):
        answer = ard.readline()
        if answer == b'ok\r\n':
            print(answer.decode('utf-8'))


def establecer_posicion(Label, x, y, z, ang_z1, ang_y, ang_z2):
    response = TPosition(x, y, z, ang_z1, ang_y, ang_z2)
    if response is not None:
        [qs, t, T] = response
        posiciones[Label] = qs
        print(f"Se estableció la posición {Label}")

def return_home():
    global ard
    ard.write("G1 A0 B0 C0 D0 X0 Y0 Z0 F50\r\n".encode('utf-8'))
    answer = ard.readline().decode('utf-8').strip()
    print(answer)
    return 0

In [ ]:
def ClawOpen(plantera):
    # Cuando está cerca de la plantera
    if plantera:
        ard.write(("M3 S300\n").encode('UTF-8'))
        answer = ard.readline().decode('utf-8').strip()
        print(answer)
    # Cuando no está cerca de la plantera
    else:
        ard.write("G1 Y0 Z0 F50\n".encode('utf-8'))
        answer = 'no'.encode('utf-8')
        while(answer != b'ok\r\n'):
            answer = ard.readline()
            if answer == b'ok\r\n':
                print(answer.decode('utf-8'))

        ard.write(("M3 S300\n").encode('UTF-8'))

        answer = ard.readline().decode('utf-8').strip()
        print(answer)

def ClawClose(plantera):
    # Cuando está cerca de la plantera
    if plantera:
        ard.write(("M3 S0\n").encode('UTF-8')) #"M3 S90\n"
        answer = ard.readline().decode('utf-8').strip()
        print(answer)
    # Cuando no está cerca de la plantera
    else:
        ard.write("G1 Y0 Z0 F50\n".encode('utf-8'))
        answer = 'no'.encode('utf-8')
        while(answer != b'ok\r\n'):
            answer = ard.readline()
            if answer == b'ok\r\n':
                print(answer.decode('utf-8'))

        ard.write(("M3 S90\n").encode('UTF-8'))

        answer = ard.readline().decode('utf-8').strip()
        print(answer)

In [ ]:
def go(Label):
    if Label == 'zero':
        return_to_zero()
    elif Label not in posiciones.keys():
        print('Label no declarado')
    else:
        enviar_trama(posiciones[Label][0], Label = Label)

# Espacio articular

In [ ]:
def solution_generation(Tn, t = np.array([0])):
    '''
    Evalua la validez de los puntos para generar un movimiento.
    Tn son las T de los puntos   Empleando las matrices Tn de las generaciones de trayectorias.
    Devuelve las qn para cada punto, el tiempo t, y el vector valid
    '''
    n = len(Tn)
    qn = np.zeros([n,4,6])
    sol = np.zeros(n)
    codo = np.zeros([n,4])
    valid = np.ones(4)

    # Espacio articular
    ang_limits = np.pi/180*np.array([[-91,91],[36,91],
                        [-91,91],[-360,360 ],
                        [-91,91 ],[-360 ,360 ]])

    for i in range(n):
        [qn[i], sol[i], codo[i]] = CineI(Tn[i]) # Aplica la cinemática inversa para cada punto


    for j in range(4):
        for k in range(6):
            if ((qn[:,j,k] < ang_limits[k,0]).any() or (qn[:,j,k] > ang_limits[k,1]).any()):
                if valid[j] == 1:
                    valid[j] = 0
                    print(f"La solucion {j+1} no se encuentra dentro del rango permitido")

    return [qn, t, valid]


# Posiciones del brazo




In [ ]:
def posicionar(clases):
    if clases==[1,0,0,0,0,0]:
        # Posicionar manzana buena
        time.sleep(5)
        ClawOpen(plantera=1)
        time.sleep(5)
        go('A')
        time.sleep(5)
        go('C1')
        time.sleep(5)
        go('C2')
        time.sleep(5)
        ClawClose(plantera=0)
        time.sleep(5)
        go('C1')
        time.sleep(5)
        go('G1')
        time.sleep(5)
        go('G2')
        time.sleep(5)
        ClawOpen(plantera=0)
        time.sleep(5)
        go('G1')
        time.sleep(5)
        go('A')
        time.sleep(5)
        go('B')
        time.sleep(5)
        ClawClose(plantera=1)

    elif clases==[0,1,0,0,0,0]:
        # Posicionar banana buena
        time.sleep(5)
        ClawOpen(plantera=1)
        time.sleep(5)
        go('A')
        time.sleep(5)
        go('C1')
        time.sleep(5)
        go('C2')
        time.sleep(5)
        ClawClose(plantera=0)
        time.sleep(5)
        go('C1')
        time.sleep(5)
        go('E1')
        time.sleep(5)
        go('E2')
        time.sleep(5)
        ClawOpen(plantera=0)
        time.sleep(5)
        go('E1')
        time.sleep(5)
        go('A')
        time.sleep(5)
        go('B')
        time.sleep(5)
        ClawClose(plantera=1)

    elif clases==[0,0,0,1,0,0]:
        # Posicionar manzana mala
        time.sleep(5)
        ClawOpen(plantera=1)
        time.sleep(5)
        go('A')
        time.sleep(5)
        go('C1')
        time.sleep(5)
        go('C2')
        time.sleep(5)
        ClawClose(plantera=0)
        time.sleep(5)
        go('C1')
        time.sleep(5)
        go('H1')
        time.sleep(5)
        go('H2')
        time.sleep(5)
        ClawOpen(plantera=0)
        time.sleep(5)
        go('H1')
        time.sleep(5)
        go('A')
        time.sleep(5)
        go('B')
        time.sleep(5)
        ClawClose(plantera=1)

    elif clases==[0,0,0,0,1,0]:
        # Posicionar banana mala
        time.sleep(5)
        ClawOpen(plantera=1)
        time.sleep(5)
        go('A')
        time.sleep(5)
        go('C1')
        time.sleep(5)
        go('C2')
        time.sleep(5)
        ClawClose(plantera=0)
        time.sleep(5)
        go('C1')
        time.sleep(5)
        go('F1')
        time.sleep(5)
        go('F2')
        time.sleep(5)
        ClawOpen(plantera=0)
        time.sleep(5)
        go('F1')
        time.sleep(5)
        go('A')
        time.sleep(5)
        go('B')
        time.sleep(5)
        ClawClose(plantera=1)


In [ ]:
def girar_plantera():
    q = posiciones['B'][0]

    # Un paso de 5 grados en cada giro
    q[5] += 5*np.pi/180

    enviar_trama(q,"Giro de plantera")



# Implementación con la VA

In [ ]:
global ard
# Se abre el puerto que controla el arduino del brazo robótico
ard = serial.Serial('COM4', 115200)
ard.close()
ard.open()

posiciones = {}

# Se predefinen las funciones de la trayectoria del robot, pasando las coordenadas de euler y aplicando la Cinemática Inversa
establecer_posicion('A',0,0,0.62415,180,0,0)
establecer_posicion('B',0.285547,0,0.088928,-90,143,0)
establecer_posicion('C1',0,0,0.62415,180,0,-10)
establecer_posicion('C2',0.281209,-0.049585,0.088928,180,143,-10)
establecer_posicion('E1',0,0,0.62415,180,0,-45)
establecer_posicion('E2',0.201913,-0.201913,0.088928,180,143,-45)
establecer_posicion('F1',0,0,0.62415,180,0,-90)
establecer_posicion('F2',0,-0.285547,0.088928,180,143,-90)
establecer_posicion('G1',0,0,0.62415,180,0,45)
establecer_posicion('G2',0.201913,0.201913,0.088928,180,143,45)
establecer_posicion('H1',0,0,0.62415,180,0,90)
establecer_posicion('H2',0,0.285547,0.088928,180,143,90)

Se estableció la posición A
La solucion 3 no se encuentra dentro del rango permitido
La solucion 4 no se encuentra dentro del rango permitido
Se estableció la posición B
Se estableció la posición C1
La solucion 3 no se encuentra dentro del rango permitido
La solucion 4 no se encuentra dentro del rango permitido
Se estableció la posición C2
Se estableció la posición E1
La solucion 3 no se encuentra dentro del rango permitido
La solucion 4 no se encuentra dentro del rango permitido
Se estableció la posición E2
Se estableció la posición F1
La solucion 3 no se encuentra dentro del rango permitido
La solucion 4 no se encuentra dentro del rango permitido
Se estableció la posición F2
Se estableció la posición G1
La solucion 3 no se encuentra dentro del rango permitido
La solucion 4 no se encuentra dentro del rango permitido
Se estableció la posición G2
Se estableció la posición H1
La solucion 3 no se encuentra dentro del rango permitido
La solucion 4 no se encuentra dentro del rango permitido

In [ ]:
go('A')

Va a la posición: A
ok



In [ ]:
# Variable encargada del conteo de "frutas detectadas"
frutas_detectadas = 0
predicted_label_ant = ''
# Se carga el modelo previamente creado
new_model = load_model('MODELO_entrenado.h5')

# Inicia stream de video
cap = cv2.VideoCapture(0)

vector = []

def vectores_label(predicted_label):
    if predicted_label == 'Manzana en buen estado':
        vector = [1, 0, 0, 0, 0, 0]
    elif predicted_label == 'Banana en buen estado':
        vector = [0, 1, 0, 0, 0, 0]
    elif predicted_label ==  'Naranja en buen estado':
        vector = [0, 0, 1, 0, 0, 0]

    elif predicted_label ==  'Manzana en mal estado':
        vector = [0, 0, 0, 1, 0, 0]

    elif predicted_label ==  'Banana en mal estado':
        vector = [0, 0, 0, 0, 1, 0]
    else:
        vector = [0, 0, 0, 0, 0, 1]
    return vector

# Estilo del rectángulo y el texto
rect_color = (255, 255, 255)  # Color del rectángulo (blanco)
text_color = (0, 0, 0)  # Color del texto (negro)
font_face = cv2.FONT_HERSHEY_TRIPLEX
font_scale = 0.7
font_thickness = 2
padding = 10

(text_width, text_height), _ = cv2.getTextSize("Predicción: Banana en buen estado", font_face, font_scale, font_thickness)
rect_x = 10
rect_y = 30
rect_w = text_width+2*padding
rect_h = text_height+2*padding

contador = 0
# Bucle principal para procesamiento en tiempo real
try:
    ########## Control de cinta ##########
    # Se abre un segundo puerto para conectarse al arduino que controla la cinta
    ard_2 = serial.Serial('COM10', 115200)
    ard_2.close()
    ard_2.open()

    time.sleep(2)
    ard_2.write("Empezar\n".encode('utf-8'))
    # Delay de 12 segundos hasta que la plantera llegue a la posición de la cámara
    time.sleep(12)
    ard_2.write("Parar\n".encode('utf-8'))

    ########## Se inicializa el brazo ##########
    time.sleep(4)
    ClawOpen(plantera=0)
    time.sleep(2)
    go('B')
    time.sleep(2)
    # Agarra la plantera para ir girando hasta encontrar las frutas
    ClawClose(plantera=1)

    while True:
        time.sleep(1)
        # Captura frame-por-frame
        ret, frame = cap.read()

        if not ret:
            print("Error al capturar imagen de la cámara.")
            break

        # Preprocesamiento de imagen
        preprocessed_image = cv2.resize(frame, (150, 150))               # Redimensiona a 150x150
        preprocessed_image = preprocessed_image/255.0                    # Normaliza los valores de píxeles
        preprocessed_image = np.expand_dims(preprocessed_image, axis=0)  # Añade un eje batch

        # Realiza la predicción
        classes = new_model.predict(preprocessed_image)

        # Muestra los resultados en el output
        print(f'PROBABILIDADES')
        print(f'Manzana en buen estado: {classes[0][0]:.6f}')
        print(f'Banana en buen estado: {classes[0][1]:.6f}')
        print(f'Naranja en buen estado: {classes[0][2]:.6f}')

        print(f'Manzana en mal estado: {classes[0][3]:.6f}')
        print(f'Banana en mal estado: {classes[0][4]:.6f}')
        print(f'Naranja en mal estado: {classes[0][5]:.6f}')

        # Determina la clase con la mayor probabilidad
        class_index = np.argmax(classes, axis=1)
        labels = ['Manzana en buen estado', 'Banana en buen estado', 'Naranja en buen estado',
                'Manzana en mal estado', 'Banana en mal estado', 'Naranja en mal estado']

        predicted_label = labels[class_index[0]]

        # Si la predicción de clase se repite de vuelta en la iteración actual, con un umbral mayor al 90 %
        if predicted_label_ant == predicted_label and classes[0][class_index] > 0.90:
            # Aumenta el conteo
            frutas_detectadas = frutas_detectadas + 1
        else:
            frutas_detectadas = 0

        # Si el conteo es mayor a 5, se asegura que acierta la clase
        if frutas_detectadas > 5:

            # Mensaje que se actualizará con la predicción
            mensaje = "Prediccion: " + predicted_label

            vector = vectores_label(predicted_label)
            print(f'Vector: {vector}')
            print(mensaje)

            posicionar(vector)
            # Se resetea el conteo
            frutas_detectadas = 0
        else:
            girar_plantera()
        # Muestra el frame resultante
        cv2.imshow('Frame', frame)
        predicted_label_ant = predicted_label
        # Presiona 'q' para salir del bucle
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

except Exception as e:
    print(f"Se ha producido un error: {e}")

finally:
    cap.release()
    cv2.destroyAllWindows()

ok

ok
Va a la posición: B
ok

ok
1/1 [==============================] - 4s 4s/step
PROBABILIDADES
Manzana en buen estado: 0.000002
Banana en buen estado: 0.999037
Naranja en buen estado: 0.000391
Manzana en mal estado: 0.000000
Banana en mal estado: 0.000566
Naranja en mal estado: 0.000003
Va a la posición: Giro de plantera
ok

1/1 [==============================] - 2s 2s/step
PROBABILIDADES
Manzana en buen estado: 0.000096
Banana en buen estado: 0.998969
Naranja en buen estado: 0.000791
Manzana en mal estado: 0.000005
Banana en mal estado: 0.000138
Naranja en mal estado: 0.000000
Va a la posición: Giro de plantera
ok

1/1 [==============================] - 1s 833ms/step
PROBABILIDADES
Manzana en buen estado: 0.000001
Banana en buen estado: 0.999676
Naranja en buen estado: 0.000006
Manzana en mal estado: 0.000000
Banana en mal estado: 0.000318
Naranja en mal estado: 0.000000
Va a la posición: Giro de plantera
ok

1/1 [==============================] - 1s 833ms/step
PROBABILIDADES
Manz

In [ ]:
ard.close()
ard_2.close()